In [3]:
import math
from typing import Sequence, List
import pandas as pd
from pathlib import Path

In [ ]:
def interp_linear(x_tab: Sequence[float], y_tab: Sequence[float], x: float) -> float:
    """Linear interpolate y(x) on ascending x_tab; clamp at ends."""
    n = len(x_tab)
    if n == 0:
        return float("nan")
    if x <= x_tab[0]:
        return float(y_tab[0])
    if x >= x_tab[-1]:
        return float(y_tab[-1])
    lo, hi = 0, n - 1
    while hi - lo > 1:
        mid = (lo + hi) // 2
        if x_tab[mid] <= x:
            lo = mid
        else:
            hi = mid
    x0, x1 = x_tab[lo], x_tab[hi]
    y0, y1 = y_tab[lo], y_tab[hi]
    if x1 == x0:
        return float(y0)
    w = (x - x0) / (x1 - x0)
    return float(y0 + w * (y1 - y0))

def shift_rating_axis(stage_axis: Sequence[float], shift_val: float) -> List[float]:
    """
    Create a *temporarily shifted* rating curve axis for a given time step.
    If original rating is Q = R(H), applying shift s means using R(H + s).
    Implement this by shifting the axis by -s: H_shifted[i] = H[i] - s.
    """
    return [float(h) - float(shift_val) for h in stage_axis]

# ---------- one-step forward model (stage -> discharge) ----------

def discharge_from_stage_at_time(
    GH_thebes_t: float,          # unshifted Thebes GH at time t
    GH_cape_t: float,            # Cape GH at time t
    shift_t: float,              # time-based shift at Thebes for time t
    rating_stage_axis: Sequence[float],   # original rating stage axis (ascending)
    rating_q_axis: Sequence[float],       # original rating discharge axis
    fall_stage_axis: Sequence[float],     # GH axis for rated fall
    fall_fr_axis: Sequence[float],        # rated fall values
    factor_ratio_axis: Sequence[float],   # r = Fm/Fr axis
    factor_val_axis: Sequence[float],     # factor values
    Ccape: float,
    Cthebes: float
) -> dict:
    # 1) Build *shifted* rating curve axis for this time and evaluate QR at raw GH
    rating_axis_t = shift_rating_axis(rating_stage_axis, shift_t)  # axis shifted by -shift
    QR = interp_linear(rating_axis_t, rating_q_axis, GH_thebes_t)

    # 2) Rated fall uses *unshifted* GH
    Fr = interp_linear(fall_stage_axis, fall_fr_axis, GH_thebes_t)

    # 3) Measured fall uses pure elevations (no shift)
    Fm = (float(GH_cape_t) + float(Ccape)) - (float(GH_thebes_t) + float(Cthebes))

    # 4) Ratio and factor
    if not math.isfinite(Fr) or Fr <= 0.0:
        r = float("nan"); factor = float("nan")
    else:
        r = Fm / Fr
        factor = interp_linear(factor_ratio_axis, factor_val_axis, r)

    # 5) Backwater-adjusted discharge
    Q_new = QR * factor if math.isfinite(QR) and math.isfinite(factor) else float("nan")

    return {
        "QR_base_cfs": QR,
        "Fr_ft": Fr,
        "Fm_ft": Fm,
        "ratio_Fm_over_Fr": r,
        "factor": factor,
        "Q_new_cfs": Q_new
    }

# ---------- driver over time series ----------

def main():
    # Load tables
    rating_df = pd.read_csv("rating_table.csv")
    fall_df   = pd.read_csv("fall_rated.csv")
    factor_df = pd.read_csv("factor_table.csv")

    # Detect stage-axis column name for the rating table
    stage_col = "GsH_ft" if "GsH_ft" in rating_df.columns else ("GH_ft" if "GH_ft" in rating_df.columns else None)
    if stage_col is None:
        raise ValueError("rating_table.csv must have a stage column named 'GsH_ft' or 'GH_ft'.")

    # Sort to ensure ascending axes
    rating_df = rating_df.sort_values(stage_col)
    fall_df   = fall_df.sort_values("GH_ft")
    factor_df = factor_df.sort_values("ratio")

    rating_stage_axis = rating_df[stage_col].astype(float).to_list()
    rating_q_axis     = rating_df["Q_cfs"].astype(float).to_list()
    fall_stage_axis   = fall_df["GH_ft"].astype(float).to_list()
    fall_fr_axis      = fall_df["Fr_ft"].astype(float).to_list()
    factor_ratio_axis = factor_df["ratio"].astype(float).to_list()
    factor_val_axis   = factor_df["factor"].astype(float).to_list()

    # Load time series
    thebes = pd.read_csv("thebes_stage.csv", parse_dates=["time"]).sort_values("time")
    cape   = pd.read_csv("cape_stage.csv",   parse_dates=["time"]).sort_values("time")
    shift  = pd.read_csv("shift_series.csv", parse_dates=["time"]).sort_values("time")

    # Align by time
    df = thebes.merge(cape, on="time", how="inner").merge(shift, on="time", how="left")
    df["shift_ft"] = df["shift_ft"].fillna(0.0)

    # Datum constants (same as your workbook)
    Ccape   = 304.27
    Cthebes = 299.70

    # Compute discharge row-by-row
    results = []
    for row in df.itertuples(index=False):
        vals = discharge_from_stage_at_time(
            GH_thebes_t       = float(row.GH_Thebes_ft),
            GH_cape_t         = float(row.Cape_GH_ft),
            shift_t           = float(row.shift_ft),
            rating_stage_axis = rating_stage_axis,
            rating_q_axis     = rating_q_axis,
            fall_stage_axis   = fall_stage_axis,
            fall_fr_axis      = fall_fr_axis,
            factor_ratio_axis = factor_ratio_axis,
            factor_val_axis   = factor_val_axis,
            Ccape             = Ccape,
            Cthebes           = Cthebes
        )
        results.append(vals)

    out = pd.concat([df.reset_index(drop=True), pd.DataFrame(results)], axis=1)

    # Save
    out.to_csv("q_from_usgs_stage.csv", index=False)
    print(f"Wrote q_from_usgs_stage.csv with {len(out)} rows")


In [ ]:
if __name__ == "__main__":
    main()